In [1]:
using DataStructures # -> declaración de datos
#some useful reminders --> https://learnxinyminutes.com/docs/julia/

In [4]:
unGrafo=generarGrafoMST(25,10000)
lista,valorFo=prim(unGrafo)
println(lista)
println(valorFo)

Bool[true, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false]	0
Bool[true, true, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false]	177
Bool[true, true, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, true, false, false, false, false]	407
Bool[true, true, false, false, false, false, false, false, false, false, false, false, true, false, false, false, false, false, false, false, true, false, false, false, false]	690
Bool[true, true, false, false, false, false, false, false, false, false, false, false, true, false, false, false, false, false, false, true, true, false, false, false, false]	1340
Bool[true, true, false, false, false, false, false, false, false, false, false, false, true, fal

# Procesos

Desde un punto de vista computacional, una de las características más significativas de los algoritmos de flujos es que nos obligan a estructurar las interacciones entre varios códigos para resolver el problema conjuntamente.

Aquí veremos un caso, flujo máximo que necesitará el desarrollo de varios bloques que compondrán el conjunto final

## Funcionamiento de caminos mínimos

In [5]:
type grafoCaminos
    vertices::Int64
    longitud::Array{Int64,2} #arco[i,j] indica la longitud
end

function generarGrafoCaminos(n::Int64,maxLongitud::Int64)
  a=Array{Int64}(n,n)
  rand!(a,1:maxLongitud)
  return grafoCaminos(n,a)
end


generarGrafoCaminos (generic function with 1 method)

In [10]:
# Algoritmo de Dijkstra
#----------------------------------
function dijkstra(grafo::grafoCaminos,origen::Int64)
    ∞=typemax(Int64)
    tr=zeros(Int64,grafo.vertices)
    l=Array{Int64}(grafo.vertices)
    fill!(l,∞)
    l[origen]=0
    Q=PriorityQueue()
    enqueue!(Q,origen,0)
    while isempty(Q)==false
        v=dequeue!(Q)
        for w in 1:grafo.vertices
            if l[w] > (l[v]+grafo.longitud[v,w])
                l[w]=l[v]+grafo.longitud[v,w]
                tr[w]=v
                if haskey(Q,w)
                    Q[w]=l[w]
                else
                    enqueue!(Q,w,l[w])
                end
            end
            assert(l[w]>(0-10000000)) #warning
        end
    end
    return l,tr
end

dijkstra2 (generic function with 1 method)

In [11]:
# Algoritmo de Bellman Ford
#---------------------------
function bellmanFord(g::grafoCaminos,origen::Int64)
  # paso 1 (inicializamos)
  ∞=typemax(Int64)
  tr=zeros(Int64,g.vertices)
  l=Array{Int64}(g.vertices)
  fill!(l,∞)
  l[origen]=0
  # paso 2 (recurrencia de programación dinámica)
  for i = 1:g.vertices-1 #número de arcos
    for v in 1:g.vertices
      for w in 1:g.vertices #dos loops para pasar por todos los arcos
        if l[w]> (l[v]+g.longitud[v,w])
          l[w]=l[v]+g.longitud[v,w]
          tr[w]=v
        end
      end
    end
  end
  # paso 3 (búsqueda de un circuito)
  for v in 1:g.vertices
    for w in 1:g.vertices #dos loops para pasar por todos los arcos
      if l[w] > (l[v]+g.longitud[v,w])
        #println("longitud: ",l[w]," vs ",l[v]+g.longitud[v,w]," y v: ",v," w: ",w)
        visited=zeros(Bool,g.vertices)
        while visited[w]==false
          visited[w]=true
          w=tr[w]
        end
        return w,l,tr
      end
    end
  end
  return 0,l,tr
end

bellmanFord (generic function with 1 method)

In [12]:
unGrafo=generarGrafoCaminos(25,10000)
unGrafo.longitud[13,14]=1; unGrafo.longitud[14,15]=1; unGrafo.longitud[15,16]=1; unGrafo.longitud[17,16]=1000; 
unGrafo.longitud[17,13]=1; unGrafo.longitud[16,17]=(-10)
conCircuito,longitud,traza=bellmanFord(unGrafo,1)
println("traza: ",traza)
# Si he encontrado un circuito, imprimirlo
if conCircuito>0
  println("arco: (",conCircuito,",",traza[conCircuito],")")
  w=traza[conCircuito]
  while w != conCircuito
    println("arco: (",w,",",traza[w],")")
    w=traza[w]
  end
end


traza: [0, 4, 8, 24, 15, 1, 6, 21, 1, 24, 24, 22, 1, 7, 21, 14, 6, 8, 6, 6, 6, 15, 24, 8, 9]


In [13]:
longitud,traza=dijkstra(unGrafo,1) #si se descomenta salta la "assertion" de circuito negativo

([0, 1698, 1147, 1404, 1249, 528, 992, 789, 1356, 909  …  1509, 1497, 1996, 992, 1101, 746, 1178, 1121, 896, 2012], [0, 4, 8, 24, 15, 1, 6, 21, 1, 24  …  14, 6, 8, 6, 6, 6, 15, 24, 8, 9])

## Funcionamiento flujo máximo

In [ ]:
type grafoFlujos
    vertices::Int64
    capacidad::Array{Int64,2} #capacidad[i,j] indica la capacidad del arco
    solucion::Array{Int64,2}  #solucion[i,j] indica el flujo del arco
end

function generarGrafoFlow(n::Int64,maxCapacidad::Int64,density::Float64)
  a=Array{Int64}(n,n)
  b=Array{Int64}(n,n)
  rand!(a,1:maxCapacidad)
  for i in 1:n
    a[i,i]=0
    for j in i+1:n
      if rand()>(2*density) #números entre 0 y 1
        a[i,j]=0
        a[j,i]=0
      else # aceptaremos flujo en un único sentido 
        if rand()<0.5
          a[i,j]=0
        else
          a[j,i]=0
        end
      end
    end
  end
  return grafoFlujos(n,a,b)
end

In [ ]:
# Implementación de Ford Fulkerson
#---------------------------------
function fordFulkerson(g::grafoFlujos,origen::Int64,destino::Int64)
  fill!(g.solucion,0) #limpiar la solución
  residual=grafoCaminos(g.vertices,zeros(Int64,g.vertices,g.vertices)) #grafo de soporte
  obj=0
  #println("g.capacidad. ",g.capacidad)
  while true 
    #1. creo el grafo residual
    fill!(residual.longitud,g.vertices+1)
    for i in 1:g.vertices
      for j in 1:g.vertices
        if g.capacidad[i,j]>0
          if g.solucion[i,j]<g.capacidad[i,j]
            residual.longitud[i,j]=1
          end
          if g.solucion[i,j]>0
            residual.longitud[j,i]=1
          end
        end
      end
    end
    #2. resuelvo Dijkstra
    #println(residual)
    longitud,traza=dijkstra(residual,origen)
    println("longitud:",longitud[destino])
    if longitud[destino]<g.vertices
      extraFlow=typemax(Int32) 
      # primero tenemos que ver el flujo máximo que podemos pasar
      c=destino
      while true
        #print("arco entre: ",traza[c]," y ",c)
        if g.capacidad[traza[c],c]>0
          #estoy añadiendo flujo
          extraFlow=min(extraFlow,g.capacidad[traza[c],c]-g.solucion[traza[c],c])
          #println(" capacidad ",g.capacidad[traza[c],c]-g.solucion[traza[c],c]," extra: ",extraFlow)
        else
          #esto eliminando flujo (redirección)
          extraFlow=min(extraFlow,g.solucion[c,traza[c]])
          #println(" backwards ",g.solucion[c,traza[c]]," extra: ",extraFlow)
        end
        c=traza[c]
        if c==origen
          break
        end
      end
      # segundo tenemos que actualizar el flujo de los arcos según flujo máximo
      c=destino
      while true
        #print("cambio en arco ",traza[c],",",c)
        if g.capacidad[traza[c],c]>0
          g.solucion[traza[c],c] += extraFlow
          #println(" added ",extraFlow)
          assert(g.solucion[traza[c],c]<=g.capacidad[traza[c],c])
        else
          g.solucion[c,traza[c]] -= extraFlow
          #println(" removed ",extraFlow)
          assert(g.solucion[traza[c],c]>=0)
        end
        c=traza[c]
        if c==origen
          break
        end
      end
      obj += extraFlow
    else
      #no podemos mejorar la solución
      return obj
    end
    #println("solucion (un paso): ",g.solucion)
  end
  return totalFlow
end

In [ ]:
gEjemplo=grafoFlujos(7,[0 1 3 0 0 0 0 ; 0 0 0 5 4 0 0 ; 0 0 0 3 0 0 0 ; 0 0 0 0 0 0 2 ; 0 0 0 0 0 2 0 ; 0 0 0 0 0 0 3 ; 0 0 0 0 0 0 0],zeros(Int64,7,7))
maxFlow=fordFulkerson(gEjemplo,1,7)
println("maxFlow: ",maxFlow)
println("solucion: ",gEjemplo.solucion)

In [ ]:
gFlujos=generarGrafoFlow(10,500,0.45)
maxFlow=fordFulkerson(gFlujos,1,10)
println("maxFlow: ",maxFlow)
println("solucion: ",gFlujos.solucion)

## Funcionamiento flujo máximo a coste mínimo

In [ ]:
type grafoMinCost
    grafoF::grafoFlujos
    coste::Array{Int64,2}  #coste[i,j] indica el coste de pasar una unidad de flujo por el arco
end

function generarGrafoMinCost(n::Int64,maxCapacidad::Int64,maxCost::Int64,density::Float64)
  a=Array{Int64}(n,n)
  b=Array{Int64}(n,n)
  c=Array{Int64}(n,n)
  rand!(a,1:maxCapacidad)
  for i in 1:n
    a[i,i]=0
    for j in i+1:n
      if rand()>(2*density) #números entre 0 y 1
        a[i,j]=0
        a[j,i]=0
      else
        if rand()<0.5
          a[i,j]=0
        else
          a[j,i]=0
        end
      end
    end
  end
  for i in 1:n
    for j in 1:n
      if a[i,j]>0
        c[i,j]=rand(1:maxCost)
      else
        c[i,j]=0
      end
    end
  end
  return grafoMinCost(grafoFlujos(n,a,b),c)
end

In [ ]:
# Implementación
#----------------------------------
function minCostMaxFlow(g::grafoMinCost,origen::Int64,destino::Int64)
  # primer paso. resolver maxflow
  maxFlow=fordFulkerson(g.grafoF,origen,destino)
  coste=0 # determinamos su coste
  for i in 1:g.grafoF.vertices
    for j in 1:g.grafoF.vertices
      if g.grafoF.solucion[i,j]>0
        coste += g.grafoF.solucion[i,j]*g.coste[i,j]
      end
    end
  end
  println("maxFlow: ",maxFlow,"\t","coste inicial: ",coste)
  # la idea ahora es la misma que e flujos máximos, pero usamos Bellman Ford para detectar un ciclo de coste negativo 
  # en el grafo
  residual=grafoCaminos(g.grafoF.vertices+1,zeros(Int64,g.grafoF.vertices+1,g.grafoF.vertices+1))
  while true #mientras mejora
    #Primero creo un grafo residual
    fill!(residual.longitud,typemax(Int32)) 
    for i in 1:g.grafoF.vertices
      for j in 1:g.grafoF.vertices
        if g.grafoF.capacidad[i,j]>0  
          if g.grafoF.solucion[i,j]<g.grafoF.capacidad[i,j]
            residual.longitud[i,j]=g.coste[i,j]
          end
          if g.grafoF.solucion[i,j]>0
            residual.longitud[j,i]=0-g.coste[i,j]
          end
        end
      end
    end
    println("residual: ",residual)
    #Luego resuelvo Bellman-Ford
    conCircuito,longitud,traza=bellmanFord(residual,origen)
    println("conCircuito: ",conCircuito," ",traza)
    #Si hay un circuito
    if conCircuito!=0
      #detecto circuito
      cMin=typemax(Int32)
      w=traza[conCircuito]
      while w != conCircuito
        if residual.longitud[traza[w],w]>0 #arco que añade
          c=g.grafoF.capacidad[traza[w],w]-g.grafoF.solucion[traza[w],w]
        else # arco que reduce
          c=g.grafoF.solucion[w,traza[w]] 
        end
        cMin=min(c,cMin)
        println(w," ",traza[w]," -> ",residual.longitud[traza[w],w],"  ** ",c)
        w=traza[w]
      end
      if residual.longitud[traza[w],w]>0 # arco que añade
        c=g.grafoF.capacidad[traza[w],w]-g.grafoF.solucion[traza[w],w]
      else # arco que reduce
        c=g.grafoF.solucion[w,traza[w]]
      end
      cMin=min(c,cMin)
      println(w," ",traza[w]," -> ",residual.longitud[traza[w],w]," -- ",c)
      # alterar el flujo en el circuito
      w=traza[conCircuito]
      while w != conCircuito
        if residual.longitud[traza[w],w]>0
          g.grafoF.solucion[traza[w],w] += cMin
          assert(g.grafoF.solucion[traza[w],w]<=g.grafoF.capacidad[traza[w],w])
          coste += cMin*g.coste[traza[w],w]
        else
          g.grafoF.solucion[w,traza[w]] -= cMin
          assert(g.grafoF.solucion[w,traza[w]]<=g.grafoF.capacidad[w,traza[w]])
          coste -= cMin*g.coste[w,traza[w]]
        end
        w=traza[w]
      end
      if residual.longitud[traza[w],w]>0
        g.grafoF.solucion[traza[w],w] += cMin
        assert(g.grafoF.solucion[traza[w],w]<=g.grafoF.capacidad[traza[w],w])
        coste += cMin*g.coste[traza[w],w]
      else # arco que reduce
        g.grafoF.solucion[w,traza[w]] -= cMin
        assert(g.grafoF.solucion[w,traza[w]]>=0)
        coste -= cMin*g.coste[w,traza[w]]
      end
      println("coste: ",coste)
    else
      return coste
    end
  end
end

In [ ]:
gMinCostEjemplo=grafoMinCost(grafoFlujos(4,[0 3 2 0; 0 0 1 0; 0 0 0 2; 0 0 0 0],zeros(Int64,4,4)),[0 10 50 0; 0 0 30 0; 0 0 0 20; 0 0 0 0])
coste=minCostMaxFlow(gMinCostEjemplo,1,4)
println("coste final: ",coste)

In [ ]:
gMinCost=generarGrafoMinCost(10,10,100,0.45)
coste=minCostMaxFlow(gMinCost,1,10)